### In this section, I will involve a LLM as an question-generator to make hypothetical questions based on the docments
#### That is one of reasons why I call the RAG system as ***Agentic-RAG***

In [17]:
import json
import uuid
import os
from typing import List
from pathlib import Path
from pydantic import BaseModel, Field

In [2]:
# This function will load the json file to json object
def load_json_list(path: str):    
    with open(path, mode = "r", encoding="utf-8") as f:
        return json.load(f)

In [3]:
data = load_json_list(r"datasets\chuncked_medicine_data.json")

In [4]:
data[:2]

[{'doc_id': '1bf5880b-93ec-4ac9-a0cb-eb35693ccce4',
  'questions': ['why is Phenylephrine prescribed?'],
  'docs': ['phenylephrine is used to relieve nasal discomfort caused by colds, allergies, and hay fever. it is also used to relieve sinus congestion and pressure. phenylephrine will relieve symptoms but will not treat the cause of the symptoms or speed recovery. phenylephrine is in a class of medications called nasal decongestants. it works by reducing swelling of the blood vessels in the nasal passages.about Phenylephrine'],
  'original_doc': 'phenylephrine is used to relieve nasal discomfort caused by colds, allergies, and hay fever. it is also used to relieve sinus congestion and pressure. phenylephrine will relieve symptoms but will not treat the cause of the symptoms or speed recovery. phenylephrine is in a class of medications called nasal decongestants. it works by reducing swelling of the blood vessels in the nasal passages.about Phenylephrine'},
 {'doc_id': 'f7ad6ffd-7176-4

In [5]:
from huggingface_hub import login

In [6]:
with open("keys.txt") as f:
    os.environ["HF_TOKEN"] = f.read().strip()

# login using env var
login(os.environ["HF_TOKEN"])

print(f"Login Huggingface so that we can access the model")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Login Huggingface so that we can access the model


### Now I need to use a local LLM to generate hypothetical question for the content.

#### I tested three models in my test notebook(That's a mess. I will never show it to you). Result is showing Below:

#### Task 1: Generate hypothetical questions for a document and output JSON
* meta-llama/Llama-3.2-1B-Instruct did OK job. very fast.
* meta-llama/Meta-Llama-3-8B-Instruct is out of control no matter how I prompt it. very slow!
* ContactDoctor/Bio-Medical-Llama-3-8B did amazing job. fast enought.

#### Task 2: Summarize the document
* meta-llama/Llama-3.2-1B-Instruct did OK job. very fast.
* meta-llama/Meta-Llama-3-8B-Instruct is out of control. very slow!
* ContactDoctor/Bio-Medical-Llama-3-8B did amazing job. fast enought.

#### So I have no any reason not to select ContactDoctor/Bio-Medical-Llama-3-8B

#### As the model provider's citation:

@misc{ContactDoctor_Bio-Medical-Llama-3-8B, author = ContactDoctor, title = {ContactDoctor-Bio-Medical: A High-Performance Biomedical Language Model}, year = {2024}, howpublished = {https://huggingface.co/ContactDoctor/Bio-Medical-Llama-3-8B}, }

Bio-Medical-Llama-3-8B model is a specialized large language model designed for biomedical applications. It is finetuned from the meta-llama/Meta-Llama-3-8B-Instruct model using a custom dataset containing over 500,000 diverse entries. These entries include a mix of synthetic and manually curated data, ensuring high quality and broad coverage of biomedical topics.

The model is trained to understand and generate text related to various biomedical fields, making it a valuable tool for researchers, clinicians, and other professionals in the biomedical domain.


In [7]:
model_id = "ContactDoctor/Bio-Medical-Llama-3-8B"

In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from transformers import BitsAndBytesConfig
import torch

#### Better quantization will speed up the inference time extrordinately.

In [14]:
def best_dtype():
    if torch.cuda.is_available():
        if torch.cuda.is_bf16_supported():
            return torch.bfloat16
        else:
            return torch.float16
        
    return torch.float32

def best_device():
    return "cuda" if torch.cuda.is_available() else "cpu"

In [15]:
# ContactDoctor/Bio-Medical-Llama-3-8B works best at bfloat16 quantization.
# In order to load a lighter LLM and still don't lose the performance.
# I will load it in 8bit and keep the dtype as bfloat16. This is a balance choise.
def load_model(model_id: str):
    bnb_cfg = None

    if best_device() == "cuda":
        bnb_cfg = BitsAndBytesConfig(
            load_in_8bit=True, # I don't want to lost LLM's performance     
            load_in_8bit_fp32_cpu_offload=False,
        )

        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            quantization_config = bnb_cfg,
            dtype = best_dtype(),
            device_map = "auto",                        
        )
        print(f"The bnb configuration: {bnb_cfg}")
    else: #CPU
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            dtype = best_dtype(),
            device_map={"":best_device()}, 
            low_cpu_mem_usage=True           
        )

    return model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token = tokenizer.eos_token

model = load_model(model_id)

print("Load tokenizer done!")

In [ ]:
# Check the model's context size which is the maximum amount of tokens we can feed

print(f"Context size: {tokenizer.model_max_length}")

In [ ]:
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    return_full_text=False,   
    )

In [ ]:
hug_pipe = HuggingFacePipeline(pipeline=pipe)

In [ ]:
question_generator = ChatHuggingFace(llm=hug_pipe)

In [ ]:
# LLM model is ready. 
# Here is the definition of the structure of output schema
class QuestionList(BaseModel):
    questions: List[str] = Field(
        ...,
        description="A list of unique, concise questions in English; each ends with a question mark."
    )

In [ ]:
structured_generator = question_generator.with_structured_output(QuestionList)

In [ ]:
# Prompt engineering can guide the LLM to generate output we expect.

# def prompt_maker(tok, doc_text: str, n: int) -> str:

#     messages = [
#         {"role": "system", "content": "You are a medical student. You like ask questions when you read clinical documents."},
#         {"role": "user", "content": (
#         f"Read the document and generate {n} clinically relevant questions.\n"    
#         "Requirements:\n" 
#         "1) The questions you generate should be from different medical perspectives.\n"
#         "2) Don't produce answers. Questions only.\n"
#         "3) Output only JSON format, for example: {'questions:': ['question1','question2',...,'question5']}.\n"
#         f"Document:\n{doc_text}\n\n"
#         "JSON:"
#         )}
#     ]

#     prompt = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

#     return prompt

prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are a medical student. You like ask questions when you read clinical documents."),
    ("user", 
     "1.From the document below, write exactly {n} unique QUESTIONS only. "
     "2.The questions you generate should be from different medical perspectives. "
     "3.Don't generate answer. "
     "4.Output only JSON format, for example: {'questions:': ['question1','question2',...,'question5']}."),    
    ("user", "Document:\n{doc}")
])

In [ ]:
# A simple chain which will feed a prompt to the generator to produce stucture json object with questions
chain = prompt | structured_generator   # output is a QuestionList pydantic object

In [ ]:
# Decide how many questions it need to generate based on the length of the document.
# From my experience, 500 to 2000 : 5 questions, 100 to 500 : 3 questions, less than 100 : 1 question.
# But it can be varied.
def decide_n_questions(txt: str):
    wc = len(txt.split())
    if wc > 500:
        return 5
    elif wc > 100:
        return 3
    else:
        return 1

In [ ]:
# Now let's iterate all medicine's docments

for content in data:
    for doc in content["docs"]:
        n = decide_n_questions(doc)
        questions_list = chain.invoke({"n": n, "doc": doc})